In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pickle
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pickle
import pandas as pd
import numpy as np
import os
import json

In [2]:
def get_exact_time_lag_fast(df, value_column, timestamp_column='timestamp_utc', lag_hours=168):
    """
    Optimized version of get_exact_time_lag using vectorized operations.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing the time series data
    value_column : str
        Name of the column containing values to be lagged
    timestamp_column : str
        Name of the column containing timestamps
    lag_hours : int
        Number of hours to look back for the lag
    
    Returns:
    --------
    pandas.Series
        Series containing the lagged values
    """
    # Calculate target timestamps
    target_timestamps = df[timestamp_column] - pd.Timedelta(hours=lag_hours)
    
    # Create a merged dataframe to find matches
    reference_df = pd.DataFrame({
        'reference_time': df[timestamp_column],
        'value': df[value_column]
    }).sort_values('reference_time')
    
    # Use searchsorted to find the insertion points
    idx = np.searchsorted(reference_df['reference_time'], target_timestamps, side='right') - 1
    
    # Handle cases where idx is -1 (target time before any reference time)
    idx = np.where(idx < 0, 0, idx)
    
    # Get the matched values
    return reference_df['value'].iloc[idx].values

In [3]:
df_bbidding = pd.read_csv('bidding_training.csv')
df_bbidding["day_ahead_price"] = df_bbidding["price_x"].rename("day_ahead_price")
df_bbidding["market_price"] = df_bbidding["price_y"].rename("market_price")
df_bbidding["national_demand"] = df_bbidding["ND"].rename("national_demand")
df_bbidding["transmission_system_demand"] = df_bbidding["TSD"].rename("transmission_system_demand")
# df_bbidding["timestamp_utc"] = pd.to_datetime(df_bbidding["timestamp_utc"])
df_bbidding['timestamp_utc'] = pd.to_datetime(df_bbidding['timestamp_utc']).dt.tz_localize('UTC')


In [4]:
# Modified code to apply to your DataFrame
df_bbidding1 = df_bbidding[["timestamp_utc","market_price","day_ahead_price","volume",
                           "settlement_period","cos_hour","cos_day","1","2","3","4","5",
                           "6","7","8","9","imbalance_price","national_demand","transmission_system_demand"]].copy()

lag_configs = {
    "market_price_lag96h": ("market_price", 96),
    "imbalance_price_lag96h": ("imbalance_price", 96),
    "volume_lag96h": ("volume", 96),
    "day_ahead_price_lag1week": ("day_ahead_price", 168)
}

for new_col, (source_col, hours) in lag_configs.items():
    df_bbidding1[new_col] = get_exact_time_lag_fast(
        df_bbidding1, 
        value_column=source_col,
        lag_hours=hours
    )

In [5]:
# dfbidding1 drop first week
df_bbidding1 = df_bbidding1.loc[df_bbidding1["timestamp_utc"] > "2021-02-26 23:00:00+00:00"]

In [19]:
# Original dataset
X = df_bbidding1[["market_price_lag96h","imbalance_price_lag96h","day_ahead_price_lag1week","volume_lag96h",
                  "cos_hour","cos_day","1","2","3","4","5","6","7","8","9","national_demand","transmission_system_demand"]].values
y = df_bbidding1["day_ahead_price"].values  # Convert to numpy

# Step 1: Split into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Step 2: Further split the training set into train and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)

# Step 3: Standardize the data (use only training data to fit the scaler)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
pickle.dump(scaler, open("MLP_day_ahead.pkl", "wb"))

In [7]:
import torch
import torch.nn as nn

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        # Layer sizes from the best trial
        layer_sizes = [256, 448, 192, 96]
        dropout_rates = [0.12338360578207397, 0.2192742565593194, 0.15708417985889997, 0.253419888887539]

        # Define the layers
        self.fc1 = nn.Linear(input_dim, layer_sizes[0])
        self.fc2 = nn.Linear(layer_sizes[0], layer_sizes[1])
        self.fc3 = nn.Linear(layer_sizes[1], layer_sizes[2])
        self.fc4 = nn.Linear(layer_sizes[2], layer_sizes[3])
        self.fc5 = nn.Linear(layer_sizes[3], 1)  # Output layer

        # Dropouts
        self.dropout1 = nn.Dropout(dropout_rates[0])
        self.dropout2 = nn.Dropout(dropout_rates[1])
        self.dropout3 = nn.Dropout(dropout_rates[2])
        self.dropout4 = nn.Dropout(dropout_rates[3])

        # Activation function (Swish)
        self.swish = nn.SiLU()
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))  # First layer with ReLU
        x = self.dropout1(x)         # First dropout
        x = self.swish(self.fc2(x))  # Second layer with Swish
        x = self.dropout2(x)         # Second dropout
        x = self.swish(self.fc3(x))  # Third layer with Swish
        x = self.dropout3(x)         # Third dropout
        x = self.swish(self.fc4(x))  # Fourth layer with Swish
        x = self.dropout4(x)         # Fourth dropout
        x = self.fc5(x)              # Output layer (no activation for raw outputs)
        return x



In [8]:
# y_train = y_train.values
# y_test = y_test.values

In [8]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Step 4: Create TensorDataset and DataLoader for training, validation, and testing
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=128, shuffle=False)
val_loader = DataLoader(dataset=val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=128, shuffle=False)

# Step 5: Initialize the model, loss function, and optimizer
input_dim = X_train_tensor.shape[1]
model = MLP(input_dim)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [9]:
import torch.nn as nn

# Define MAE loss
mae_loss = nn.L1Loss()

# Training parameters
num_epochs = 500
patience = 15  # Number of epochs to wait for improvement
min_relative_improvement = 0.0001  # 0.1% improvement threshold
best_val_loss = float('inf')
early_stopping_counter = 0

# Lists to store metrics
train_losses = []
val_losses = []
test_losses = []

for epoch in range(num_epochs):
    # Training phase
    model.train()
    epoch_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = mae_loss(outputs, y_batch)  # Use MAE loss
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    avg_train_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    
    # Validation phase
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        val_loss = mae_loss(val_outputs, y_val_tensor).item()  # Use MAE loss
        val_losses.append(val_loss)
        
        test_outputs = model(X_test_tensor)
        test_loss = mae_loss(test_outputs, y_test_tensor).item()  # Use MAE loss
        test_losses.append(test_loss)
    
    # Early stopping and relative improvement check
    if val_loss < best_val_loss:
        relative_improvement = (best_val_loss - val_loss) / best_val_loss
        if relative_improvement < min_relative_improvement and epoch > 10:
            print(f"Stopping due to small relative improvement: {relative_improvement:.6f}")
            break
        
        best_val_loss = val_loss
        early_stopping_counter = 0
        # Save best model
        torch.save(model.state_dict(), 'MLP_day_ahead_price.pth')

    else:
        early_stopping_counter += 1
    
    # Early stopping check
    if early_stopping_counter >= patience:
        print(f"Early stopping triggered after {epoch + 1} epochs")
        break
    
    # Print progress
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], "
              f"Train Loss: {avg_train_loss:.4f}, "
              f"Val Loss: {val_loss:.4f}, "
              f"Test Loss: {test_loss:.4f}")

# Load best model after training
model.load_state_dict(torch.load('MLP_day_ahead_price.pth'))

# Final evaluation
model.eval()
with torch.no_grad():
    final_test_outputs = model(X_test_tensor)
    final_test_loss = mae_loss(final_test_outputs, y_test_tensor).item()  # Use MAE loss
print(f"Final Test Loss: {final_test_loss:.4f}")


Epoch [10/500], Train Loss: 45.1104, Val Loss: 63.1821, Test Loss: 44.6255
Epoch [20/500], Train Loss: 40.8001, Val Loss: 55.3947, Test Loss: 29.7798
Epoch [30/500], Train Loss: 39.2996, Val Loss: 50.9989, Test Loss: 25.3813
Epoch [40/500], Train Loss: 37.9786, Val Loss: 46.0362, Test Loss: 22.8068
Epoch [50/500], Train Loss: 37.4146, Val Loss: 43.5941, Test Loss: 21.7969
Epoch [60/500], Train Loss: 37.0843, Val Loss: 42.5782, Test Loss: 21.4924
Epoch [70/500], Train Loss: 36.6306, Val Loss: 42.0624, Test Loss: 20.9457
Epoch [80/500], Train Loss: 36.5896, Val Loss: 41.3596, Test Loss: 20.4508
Epoch [90/500], Train Loss: 36.2138, Val Loss: 41.2903, Test Loss: 20.5799
Epoch [100/500], Train Loss: 36.1325, Val Loss: 41.5075, Test Loss: 20.9258
Epoch [110/500], Train Loss: 35.9485, Val Loss: 41.8853, Test Loss: 21.3270
Epoch [120/500], Train Loss: 35.6191, Val Loss: 41.0826, Test Loss: 19.9158
Epoch [130/500], Train Loss: 35.4678, Val Loss: 40.9737, Test Loss: 20.3971
Epoch [140/500], Trai

C:\Users\paulh\AppData\Local\Temp\ipykernel_34804\2683471474.py:72: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('MLP_day_ahead_price.pth')

In [10]:
current_dir = os.getcwd()

path_df = os.path.abspath(os.path.join(current_dir, '..', 'basic_files'))
df_total_solar = pd.read_csv(os.path.join(path_df, 'solar_total_production.csv'))
df_total_solar.generation_mw = df_total_solar.generation_mw *0.5
df_total_wind = pd.read_csv(os.path.join(path_df, 'wind_total_production.csv'))
df_total_wind.generation_mw = df_total_wind.generation_mw *0.5 - df_total_wind.boa
df_imbalance_price = pd.read_csv(os.path.join(path_df, 'imbalance_price.csv'))
df_day_ahead_price = pd.read_csv(os.path.join(path_df, 'day_ahead_price.csv'))
df_market_price = pd.read_csv(os.path.join(path_df, 'market_index.csv'))
path_df = os.path.abspath(os.path.join(current_dir, '..'))
df_demand = pd.read_csv(os.path.join(path_df, 'day_ahead_demand_forecast.csv'))
df_demand = df_demand.groupby("timestamp_utc").last().reset_index()

# Get the path to the 'logs' directory in the parent directory
path = os.path.abspath(os.path.join(current_dir, '..', 'logs'))
files = os.listdir(path)
txt_files = [file for file in files if file.endswith('.txt')]
data = []
for file in txt_files:
    with open(os.path.join(path, file), 'r') as f:
        try:
            json_data = json.load(f)
            data.append(json_data)
        except json.JSONDecodeError:
            print(f"Failed to decode JSON from file: {file}")
date_name = []
for i in range(len(data)):
    date_name.append(data[i]["prediction_date"])

In [11]:
# Extrahiere Daten
dataframe_list = []

for entry in data:
    prediction_date = entry['prediction_date']
    
    # Iteriere durch jedes 'submission' Element
    for submission in entry['solution']['submission']:
        timestamp = submission['timestamp']
        probabilistic_forecast = submission['probabilistic_forecast']
        
        # Extrahiere die Werte von 'probabilistic_forecast' und füge sie der Liste hinzu
        row = {
            'prediction_date': prediction_date,
            'timestamp': timestamp,
            '1': probabilistic_forecast.get('10', None),
            '2': probabilistic_forecast.get('20', None),
            '3': probabilistic_forecast.get('30', None),
            '4': probabilistic_forecast.get('40', None),
            '5': probabilistic_forecast.get('50', None),
            '6': probabilistic_forecast.get('60', None),
            '7': probabilistic_forecast.get('70', None),
            '8': probabilistic_forecast.get('80', None),
            '9': probabilistic_forecast.get('90', None)
        }
        dataframe_list.append(row)

# Erstelle DataFrame
df_api_new = pd.DataFrame(dataframe_list)
df_api_new = df_api_new.groupby("timestamp").last().reset_index()
df_api_new

,timestamp,prediction_date,1,2,3,4,5,6,7,8,9
0,2024-10-03T22:00:00+00:00,2024-10-04,1355,487,963,1544,1330,867,326,1402,1357
1,2024-10-03T22:30:00+00:00,2024-10-04,788,584,1148,1320,1007,565,923,797,487
2,2024-10-03T23:00:00+00:00,2024-10-04,571,748,1001,1460,740,1017,1533,597,991
3,2024-10-03T23:30:00+00:00,2024-10-04,1476,1194,1512,334,1054,1572,744,1497,463
4,2024-10-04T00:00:00+00:00,2024-10-04,1352,1390,912,662,1505,1093,814,487,1340
...,...,...,...,...,...,...,...,...,...,...,...
1101,2024-10-31T20:30:00+00:00,2024-10-31,115,146,178,201,218,245,278,315,378
1102,2024-10-31T21:00:00+00:00,2024-10-31,124,158,191,211,228,255,289,325,388
1103,2024-10-31T21:30:00+00:00,2024-10-31,116,161,192,213,230,257,291,327,391
1104,2024-10-31T22:00:00+00:00,2024-10-31,118,156,193,215,233,259,292,330,393


In [12]:
from datetime import datetime, timedelta
df_api_new = df_api_new.rename(columns={
    "timestamp": "datetime"
})
df_api_new["datetime"] = pd.to_datetime(df_api_new["datetime"])
min_date = df_api_new.datetime.min() - timedelta(minutes=30)
datetimes = pd.date_range(end=min_date, periods=336, freq='30min')
datetimes = pd.date_range(end=min_date, periods=336, freq='30min')
df_half_hourly = pd.DataFrame({"datetime": datetimes})
df_half_hourly["datetime"] = pd.to_datetime(df_half_hourly["datetime"])
df_api_new = pd.merge(df_half_hourly, df_api_new, left_on='datetime', right_on='datetime', how='outer')
df_api_new

,datetime,prediction_date,1,2,3,4,5,6,7,8,9
0,2024-09-26 22:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-09-26 22:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-09-26 23:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-09-26 23:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-09-27 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1437,2024-10-31 20:30:00+00:00,2024-10-31,115.0,146.0,178.0,201.0,218.0,245.0,278.0,315.0,378.0
1438,2024-10-31 21:00:00+00:00,2024-10-31,124.0,158.0,191.0,211.0,228.0,255.0,289.0,325.0,388.0
1439,2024-10-31 21:30:00+00:00,2024-10-31,116.0,161.0,192.0,213.0,230.0,257.0,291.0,327.0,391.0
1440,2024-10-31 22:00:00+00:00,2024-10-31,118.0,156.0,193.0,215.0,233.0,259.0,292.0,330.0,393.0


In [13]:
df_demand = df_demand[["timestamp_utc","transmission_system_demand","national_demand"]]
df_demand = df_demand.rename(columns={'timestamp_utc': 'timestamp_utc_demand'})

In [14]:
df_day_ahead_price.timestamp_utc = pd.to_datetime(df_day_ahead_price.timestamp_utc)
df_market_price.timestamp_utc = pd.to_datetime(df_market_price.timestamp_utc)
df_imbalance_price.timestamp_utc = pd.to_datetime(df_imbalance_price.timestamp_utc)
df_demand.timestamp_utc_demand = pd.to_datetime(df_demand.timestamp_utc_demand)
df_api_new_merged = pd.merge(df_api_new,df_day_ahead_price, left_on='datetime', right_on='timestamp_utc', how='left')
df_api_new_merged = pd.merge(df_api_new_merged,df_market_price, left_on='datetime', right_on='timestamp_utc', how='left')
df_api_new_merged = pd.merge(df_api_new_merged,df_imbalance_price, left_on='datetime', right_on='timestamp_utc', how='left')
df_api_new_merged = pd.merge(df_api_new_merged,df_demand, left_on='datetime', right_on='timestamp_utc_demand', how='left',suffixes=('_x', '_y'))
df_api_new_merged["day_ahead_price"] = df_api_new_merged["price_x"].rename("day_ahead_price")
df_api_new_merged["market_price"] = df_api_new_merged["price_y"].rename("market_price")
df_api_new_merged["settlement_period"] = df_api_new_merged["settlement_period_x"].rename("settlement_period")
df_api_new_merged["cos_hour"] = np.cos(2*np.pi*df_api_new_merged["datetime"].dt.hour/24)
df_api_new_merged["cos_day"] = np.cos(2*np.pi*df_api_new_merged["datetime"].dt.day/7)
df_api_new_merged

,datetime,prediction_date,1,2,3,4,5,6,7,8,...,settlement_date,settlement_period,imbalance_price,timestamp_utc_demand,transmission_system_demand,national_demand,day_ahead_price,market_price,cos_hour,cos_day
0,2024-09-26 22:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2024-09-26,47.0,68.929966,NaT,NaN,NaN,7.20,21.21,0.866025,-0.222521
1,2024-09-26 22:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2024-09-26,48.0,-28.480000,NaT,NaN,NaN,7.20,3.46,0.866025,-0.222521
2,2024-09-26 23:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2024-09-27,1.0,-19.020000,NaT,NaN,NaN,3.87,3.15,0.965926,-0.222521
3,2024-09-26 23:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2024-09-27,2.0,-18.000000,NaT,NaN,NaN,3.87,-3.68,0.965926,-0.222521
4,2024-09-27 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2024-09-27,3.0,56.450000,NaT,NaN,NaN,0.45,-0.52,1.000000,0.623490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1451,2024-10-31 20:30:00+00:00,2024-10-31,115.0,146.0,178.0,201.0,218.0,245.0,278.0,315.0,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,0.500000,-0.900969
1452,2024-10-31 21:00:00+00:00,2024-10-31,124.0,158.0,191.0,211.0,228.0,255.0,289.0,325.0,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,0.707107,-0.900969
1453,2024-10-31 21:30:00+00:00,2024-10-31,116.0,161.0,192.0,213.0,230.0,257.0,291.0,327.0,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,0.707107,-0.900969
1454,2024-10-31 22:00:00+00:00,2024-10-31,118.0,156.0,193.0,215.0,233.0,259.0,292.0,330.0,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,0.866025,-0.900969


In [15]:
df_api_new_merged1 = df_api_new_merged[["datetime","market_price","day_ahead_price","volume","settlement_period","cos_hour","cos_day","1","2","3","4","5","6","7","8","9","imbalance_price","national_demand","transmission_system_demand"]].copy()
df_api_new_merged1.loc[:,"market_price_lag96h"] = df_api_new_merged1["market_price"].shift(192)
df_api_new_merged1.loc[:,"imbalance_price_lag96h"] = df_api_new_merged1["imbalance_price"].shift(192)
df_api_new_merged1.loc[:,"day_ahead_price_lag1week"] = df_api_new_merged1["day_ahead_price"].shift(336)
df_api_new_merged1.loc[:,"volume_lag96h"] = df_api_new_merged1["volume"].shift(192)
df_api_new_merged1.dropna(inplace=True)
df_api_new_merged1

,datetime,market_price,day_ahead_price,volume,settlement_period,cos_hour,cos_day,1,2,3,...,7,8,9,imbalance_price,national_demand,transmission_system_demand,market_price_lag96h,imbalance_price_lag96h,day_ahead_price_lag1week,volume_lag96h
336,2024-10-03 22:00:00+00:00,71.32,78.47,1322.45,47.0,0.866025,-0.900969,1355.0,487.0,963.0,...,326.0,1402.0,1357.0,61.00,23902.0,24402.0,12.85,-1.4600,7.20,1457.25
337,2024-10-03 22:30:00+00:00,71.42,78.47,1035.65,48.0,0.866025,-0.900969,788.0,584.0,1148.0,...,923.0,797.0,487.0,50.00,23100.0,23600.0,-3.95,-1.4600,7.20,1367.60
338,2024-10-03 23:00:00+00:00,66.94,71.36,1378.55,1.0,0.965926,-0.900969,571.0,748.0,1001.0,...,1533.0,597.0,991.0,93.50,22000.0,22500.0,-2.74,5.0000,3.87,1473.20
339,2024-10-03 23:30:00+00:00,60.03,71.36,1539.90,2.0,0.965926,-0.900969,1476.0,1194.0,1512.0,...,744.0,1497.0,463.0,93.50,21872.0,22372.0,3.85,5.0000,3.87,1799.05
340,2024-10-04 00:00:00+00:00,65.21,72.80,1818.40,3.0,1.000000,-0.900969,1352.0,1390.0,912.0,...,814.0,487.0,1340.0,93.00,21551.0,22051.0,0.03,-2.4304,0.45,1689.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156,2024-10-25 19:30:00+00:00,99.81,96.94,1717.85,42.0,0.258819,-0.900969,0.0,26.0,57.0,...,160.0,197.0,274.0,74.75,31108.0,32390.0,2.71,2.7100,85.55,1830.75
1157,2024-10-25 20:00:00+00:00,99.47,88.68,2160.55,43.0,0.500000,-0.900969,0.0,34.0,68.0,...,175.0,221.0,313.0,140.00,30161.0,31344.0,-9.74,-9.7400,85.55,2016.45
1158,2024-10-25 20:30:00+00:00,97.96,88.68,1620.65,44.0,0.500000,-0.900969,0.0,27.0,58.0,...,173.0,211.0,282.0,72.50,28806.0,30011.0,5.26,5.2600,78.24,1717.05
1159,2024-10-25 21:00:00+00:00,94.28,99.80,1585.70,45.0,0.707107,-0.900969,0.0,16.0,49.0,...,162.0,201.0,266.0,129.50,27452.0,28513.0,-6.69,-6.6900,78.24,1755.00


In [16]:
X_test = df_api_new_merged1[["market_price_lag96h","imbalance_price_lag96h","day_ahead_price_lag1week","volume_lag96h",
                    "cos_hour","cos_day","1","2","3","4","5","6","7","8","9","national_demand","transmission_system_demand"]].values
X_test = scaler.transform(X_test)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

model.eval()
with torch.no_grad():
    final_test_outputs = model(X_test_tensor)
    final_test_outputs = final_test_outputs.numpy().flatten()

df_api_new_merged1["day_ahead_price_predictions"] = final_test_outputs
df_api_new_merged1

,datetime,market_price,day_ahead_price,volume,settlement_period,cos_hour,cos_day,1,2,3,...,8,9,imbalance_price,national_demand,transmission_system_demand,market_price_lag96h,imbalance_price_lag96h,day_ahead_price_lag1week,volume_lag96h,day_ahead_price_predictions
336,2024-10-03 22:00:00+00:00,71.32,78.47,1322.45,47.0,0.866025,-0.900969,1355.0,487.0,963.0,...,1402.0,1357.0,61.00,23902.0,24402.0,12.85,-1.4600,7.20,1457.25,34.887680
337,2024-10-03 22:30:00+00:00,71.42,78.47,1035.65,48.0,0.866025,-0.900969,788.0,584.0,1148.0,...,797.0,487.0,50.00,23100.0,23600.0,-3.95,-1.4600,7.20,1367.60,22.877462
338,2024-10-03 23:00:00+00:00,66.94,71.36,1378.55,1.0,0.965926,-0.900969,571.0,748.0,1001.0,...,597.0,991.0,93.50,22000.0,22500.0,-2.74,5.0000,3.87,1473.20,22.675936
339,2024-10-03 23:30:00+00:00,60.03,71.36,1539.90,2.0,0.965926,-0.900969,1476.0,1194.0,1512.0,...,1497.0,463.0,93.50,21872.0,22372.0,3.85,5.0000,3.87,1799.05,41.155598
340,2024-10-04 00:00:00+00:00,65.21,72.80,1818.40,3.0,1.000000,-0.900969,1352.0,1390.0,912.0,...,487.0,1340.0,93.00,21551.0,22051.0,0.03,-2.4304,0.45,1689.75,30.219080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156,2024-10-25 19:30:00+00:00,99.81,96.94,1717.85,42.0,0.258819,-0.900969,0.0,26.0,57.0,...,197.0,274.0,74.75,31108.0,32390.0,2.71,2.7100,85.55,1830.75,83.288284
1157,2024-10-25 20:00:00+00:00,99.47,88.68,2160.55,43.0,0.500000,-0.900969,0.0,34.0,68.0,...,221.0,313.0,140.00,30161.0,31344.0,-9.74,-9.7400,85.55,2016.45,81.113098
1158,2024-10-25 20:30:00+00:00,97.96,88.68,1620.65,44.0,0.500000,-0.900969,0.0,27.0,58.0,...,211.0,282.0,72.50,28806.0,30011.0,5.26,5.2600,78.24,1717.05,76.427040
1159,2024-10-25 21:00:00+00:00,94.28,99.80,1585.70,45.0,0.707107,-0.900969,0.0,16.0,49.0,...,201.0,266.0,129.50,27452.0,28513.0,-6.69,-6.6900,78.24,1755.00,75.792381


In [17]:
#plot imbalance price and predictions
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_api_new_merged1["datetime"], y=df_api_new_merged1["day_ahead_price"], mode='lines', name='day_ahead_price'))
fig.add_trace(go.Scatter(x=df_api_new_merged1["datetime"], y=df_api_new_merged1["day_ahead_price_predictions"], mode='lines', name='day_ahead_price Predictions'))
fig.update_layout(title='day_ahead_price and Predictions', xaxis_title='Timestamp', yaxis_title='day_ahead_price Price')
fig.show()

In [18]:
#MAE calculation for imbalance price predictions
mae = np.mean(np.abs(df_api_new_merged1["day_ahead_price"] - df_api_new_merged1["day_ahead_price_predictions"]))
print(f"MAE for day_ahead_price predictions: {mae:.4f}")

MAE for day_ahead_price predictions: 28.2869


In [19]:
df_api_new_merged1.drop(columns=["datetime"]).corr()["day_ahead_price"]

market_price                   0.459050
day_ahead_price                1.000000
volume                         0.023106
settlement_period              0.237255
cos_hour                      -0.332983
cos_day                       -0.124310
1                             -0.084179
2                             -0.124610
3                             -0.120535
4                             -0.126306
5                             -0.131031
6                             -0.115995
7                             -0.120230
8                             -0.105632
9                             -0.107018
imbalance_price                0.572484
market_price_lag96h           -0.013696
imbalance_price_lag96h         0.100072
day_ahead_price_lag1week       0.128432
volume_lag96h                  0.139725
day_ahead_price_predictions    0.257556
Name: day_ahead_price, dtype: float64

In [20]:
df_api_new_merged1.columns

Index(['datetime', 'market_price', 'day_ahead_price', 'volume',
       'settlement_period', 'cos_hour', 'cos_day', '1', '2', '3', '4', '5',
       '6', '7', '8', '9', 'imbalance_price', 'market_price_lag96h',
       'imbalance_price_lag96h', 'day_ahead_price_lag1week', 'volume_lag96h',
       'day_ahead_price_predictions'],
      dtype='object')

In [21]:
df_day_ahead_demand = pd.read_csv('D:/Users/paulh\Desktop/Domäneprojekt2/Energy_production_price_prediction/day_ahead_demand_forecast.csv')
df_margin_forecast = pd.read_csv('D:/Users/paulh\Desktop/Domäneprojekt2/Energy_production_price_prediction/margin_forecast.csv')
df_margin_forecast.forecast_date = pd.to_datetime(df_margin_forecast.forecast_date)
df_day_ahead_demand.timestamp_utc = pd.to_datetime(df_day_ahead_demand.timestamp_utc)

In [22]:
df_api_new_merged2 = pd.merge(df_api_new_merged1,df_day_ahead_demand, left_on='datetime', right_on='timestamp_utc', how='left')
df_api_new_merged2["date"] = df_api_new_merged2["datetime"].dt.date
df_api_new_merged2["date"] = pd.to_datetime(df_api_new_merged2["date"])
df_api_new_merged2 = pd.merge(df_api_new_merged2,df_margin_forecast, left_on='date', right_on='forecast_date', how='left')

In [23]:
df_api_new_merged2.dropna(inplace=True)
df_api_new_merged2 = df_api_new_merged2.groupby("timestamp_utc").last().reset_index()
df_api_new_merged2

,timestamp_utc,datetime,market_price,day_ahead_price,volume,settlement_period_x,cos_hour,cos_day,1,2,...,settlement_date,settlement_period_y,boundary,publish_time_utc_x,transmission_system_demand,national_demand,date,forecast_date,publish_time_utc_y,margin
0,2024-10-04 00:00:00+00:00,2024-10-04 00:00:00+00:00,65.21,72.80,1818.40,3.0,1.000000,-0.900969,1352.0,1390.0,...,2024-10-04,3,N,2024-10-03T20:45:00Z,22051.0,21551.0,2024-10-04,2024-10-04,2024-10-02T15:00:00Z,9842.0
1,2024-10-04 00:30:00+00:00,2024-10-04 00:30:00+00:00,64.79,72.80,1784.30,4.0,1.000000,-0.900969,1279.0,410.0,...,2024-10-04,4,N,2024-10-03T20:45:00Z,21886.0,21386.0,2024-10-04,2024-10-04,2024-10-02T15:00:00Z,9842.0
2,2024-10-04 01:00:00+00:00,2024-10-04 01:00:00+00:00,64.26,65.30,1647.20,5.0,0.965926,-0.900969,1223.0,378.0,...,2024-10-04,5,N,2024-10-04T00:46:00Z,21486.0,20986.0,2024-10-04,2024-10-04,2024-10-02T15:00:00Z,9842.0
3,2024-10-04 01:30:00+00:00,2024-10-04 01:30:00+00:00,65.61,65.30,1406.60,6.0,0.965926,-0.900969,848.0,1046.0,...,2024-10-04,6,N,2024-10-04T00:46:00Z,21471.0,20971.0,2024-10-04,2024-10-04,2024-10-02T15:00:00Z,9842.0
4,2024-10-04 02:00:00+00:00,2024-10-04 02:00:00+00:00,58.19,71.46,1149.45,7.0,0.866025,-0.900969,1469.0,543.0,...,2024-10-04,7,N,2024-10-04T00:46:00Z,20947.0,20447.0,2024-10-04,2024-10-04,2024-10-02T15:00:00Z,9842.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661,2024-10-21 19:30:00+00:00,2024-10-21 19:30:00+00:00,71.47,79.79,2221.80,42.0,0.258819,1.000000,263.0,343.0,...,2024-10-21,42,N,2024-10-21T18:15:00Z,32190.0,30136.0,2024-10-21,2024-10-21,2024-10-19T21:00:00Z,16904.0
662,2024-10-21 20:00:00+00:00,2024-10-21 20:00:00+00:00,87.87,77.99,2156.50,43.0,0.500000,1.000000,266.0,345.0,...,2024-10-21,43,N,2024-10-21T18:15:00Z,30604.0,28749.0,2024-10-21,2024-10-21,2024-10-19T21:00:00Z,16904.0
663,2024-10-21 20:30:00+00:00,2024-10-21 20:30:00+00:00,86.70,77.99,2183.15,44.0,0.500000,1.000000,257.0,341.0,...,2024-10-21,44,N,2024-10-21T18:15:00Z,29396.0,27437.0,2024-10-21,2024-10-21,2024-10-19T21:00:00Z,16904.0
664,2024-10-21 21:00:00+00:00,2024-10-21 21:00:00+00:00,71.16,74.71,1754.15,45.0,0.707107,1.000000,249.0,338.0,...,2024-10-21,45,N,2024-10-21T20:45:00Z,27294.0,26052.0,2024-10-21,2024-10-21,2024-10-19T21:00:00Z,16904.0


In [21]:
#prediction for the whole dataset
X = df_bbidding1[["market_price_lag96h","imbalance_price_lag96h","day_ahead_price_lag1week","volume_lag96h",
                  "cos_hour","cos_day","1","2","3","4","5","6","7","8","9","national_demand","transmission_system_demand"]].values
X = scaler.transform(X)
X_tensor = torch.tensor(X, dtype=torch.float32)

model.eval()
with torch.no_grad():
    final_outputs = model(X_tensor)
    final_outputs = final_outputs.numpy().flatten()

df_bbidding1["day_ahead_price_predictions"] = final_outputs
df_bbidding1

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period,cos_hour,cos_day,1,2,3,...,8,9,imbalance_price,national_demand,transmission_system_demand,market_price_lag96h,imbalance_price_lag96h,volume_lag96h,day_ahead_price_lag1week,day_ahead_price_predictions
146,2021-02-28 00:00:00+00:00,55.58,53.03,497.60,1,1.000000,0.527078,-0.526708,-0.090521,-9.619702e-09,...,6.736776,17.888765,33.65,25638,27270,-9.71,7.00,900.35,17.34,41.925751
147,2021-02-28 00:30:00+00:00,57.95,53.03,554.50,2,1.000000,0.527078,-0.526708,-0.090521,-9.808374e-09,...,6.736776,17.888765,62.70,25983,27557,-9.71,7.00,900.35,17.34,41.631996
148,2021-02-28 01:00:00+00:00,53.11,47.03,739.95,3,0.965926,0.527078,-0.702233,-0.090521,-9.808374e-09,...,6.736776,17.861359,34.10,25716,27176,-9.71,7.00,900.35,14.25,40.717239
149,2021-02-28 01:30:00+00:00,48.58,47.03,360.45,4,0.965926,0.527078,-0.702233,-0.090521,-9.808374e-09,...,6.736776,17.863510,33.50,24984,26794,-9.71,7.00,900.35,14.25,41.249939
150,2021-02-28 02:00:00+00:00,47.27,42.32,422.10,5,0.866025,0.527078,-0.702233,-0.090521,-1.125217e-08,...,6.736776,17.863510,35.05,24533,26652,-9.71,7.00,900.35,13.69,41.067253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42280,2023-08-25 23:30:00+00:00,83.91,84.41,1201.85,2,0.965926,-0.591261,57.607621,79.899250,9.364298e+01,...,215.222854,265.101297,66.01,20158,24583,91.88,126.00,1271.20,50.21,78.457970
42281,2023-08-26 00:00:00+00:00,82.13,79.96,1066.95,3,1.000000,-0.577292,29.568620,54.005683,6.902610e+01,...,173.525543,229.298724,66.01,18726,20708,90.64,90.64,1643.15,37.82,79.850082
42282,2023-08-26 00:30:00+00:00,78.76,79.96,997.60,4,1.000000,-0.577292,25.170915,48.936644,6.365008e+01,...,167.753710,224.068561,66.01,18445,20326,95.15,115.99,1599.25,37.82,80.566200
42283,2023-08-26 01:00:00+00:00,77.47,73.84,913.25,5,0.965926,-0.577292,23.416983,41.031986,5.687607e+01,...,150.622068,213.537817,66.01,18185,20582,91.30,125.00,1619.70,31.66,75.491852


In [22]:
df_bbidding1.to_csv('bidding_training_predictions_day_ahead_price.csv', index=False)

In [24]:
tesz

NameError: name 'tesz' is not defined